# 1/ OLIVIER_IT ⋈ 6SIGMA (It_equipment) ⋈ CMDB

Vérification de la présence des equipements Olivier_it simultanément sur 6Sigma et sur la CMDB :
    
    - Controle de l'asset_id
    - Controle de Position 
    - COntrole de la hauteur

`IT_Equipment_v2` : Table IT_Equipment provenant d'un fichier csv

In [ ]:
DROP TABLE IF EXISTS `IT_Equipment_v2` ;
CREATE TABLE IT_Equipment_v2
as
SELECT * FROM `IT_Equipment`

`assets_filtree_v2` : Table assets_new (CMDB) filtré aux deux localisations : FRVLB0C2 / FRTCRAVA

In [ ]:
DROP TABLE IF EXISTS `assets_filtree_v2` ;
CREATE TABLE assets_filtree_v2
as
select *
from `assets_new`
where (API = 'FRVLB0C2' or API = 'FRTCRAVA') ;

In [ ]:
ALTER TABLE assets_filtree_v2
ADD COLUMN Rack varchar(255),
ADD COLUMN Slot varchar(255),
ADD COLUMN Slot_begin varchar(255),
ADD COLUMN Slot_end varchar(255);

In [ ]:
UPDATE assets_filtree_v2
SET 
	Rack = SUBSTRING_INDEX(DALLE, '/', 1)  ,
	Slot = SUBSTRING_INDEX(DALLE, '/', -1) ,
	Slot_begin = SUBSTRING_INDEX(SUBSTRING_INDEX(DALLE, '/', -1), '-', 1) ,
	Slot_end = SUBSTRING_INDEX(DALLE, '-', -1)

In [ ]:
DROP TABLE IF EXISTS `assets_filtree_reduced` ;
CREATE TABLE assets_filtree_reduced
AS
SELECT ype, subtype, ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE, SALLE, Rack, Slot_Begin, Slot_End, No_serie, etat, api, tag_partenaire
from `assets_filtree_v2` aa
WHERE SALLE in ('P1', 'P2', 'P3', 'P4', '0')

In [ ]:
DROP TABLE IF EXISTS `assets_filtree_reduced_not_retired` ;
CREATE TABLE assets_filtree_reduced_not_retired
AS
SELECT ype, subtype, ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE, SALLE, Rack, Slot_Begin, Slot_End, No_serie, etat, api, tag_partenaire
from `assets_filtree_v2` aa
WHERE SALLE in ('P1', 'P2', 'P3', 'P4')

##### On ajoute la même forme de position que sur 6sigma et olivier_it pour fluidifier les query

In [ ]:

ALTER TABLE `assets_filtree_reduced_not_retired`
ADD COLUMN reperage_id varchar(255) ;

update `assets_filtree_reduced_not_retired`
set reperage_id = concat(Rack,":Slot ", slot_begin)


`IT_Equipment_match_position_reduced_v3` : groupement des lignes par positions et affichage d'un seul asset_id par positions + nombres d'equipements sur la même position

DROP VIEW IF EXISTS IT_Equipment_match_position_reduced_v3_not_retired;
CREATE VIEW IT_Equipment_match_position_reduced_v3_not_retired
as
SELECT it.nom_de_la_salle, it.reperage_id, count(it.reperage_id) as RowNumber,
		concat(aa.asset_id) asset_id

FROM   `olivier_it` it  
INNER JOIN `assets_filtree_reduced_not_retired`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
GROUP BY it.reperage_id

In [ ]:
ALTER TABLE olivier_it 
ADD COLUMN status_6Sigma varchar(255),
ADD COLUMN status_CMDB varchar(255),
ADD COLUMN status_position_6Sigma varchar(255),
ADD COLUMN status_position_CMDB varchar(255),
ADD COLUMN status_hauteur varchar(255);

## Status d'asset_id et de position

In [ ]:
-- 1537
UPDATE `olivier_it` it 
INNER JOIN `IT_Equipment_records` it2 ON it.ASSET_ID = it2.ASSET_ID 

SET it.status_6Sigma = 'OK'

WHERE (it.ASSET_ID != '' or it.ASSET_ID != '' or it2.ASSET_ID != '' or it2.ASSET_ID != '') and it2.INSERTED = (select * from `last_date_insertion`)

In [ ]:
-- 1441
UPDATE `olivier_it` it 
INNER JOIN `assets_filtree_reduced_not_retired` it2 ON it.ASSET_ID = it2.ASSET_ID 

SET it.status_cmdb = 'OK'

WHERE (it.ASSET_ID != '' or it.ASSET_ID != '' or it2.ASSET_ID != '' or it2.ASSET_ID != '')

In [ ]:
-- 469
UPDATE `olivier_it` it 
SET status_6Sigma = 'KO'
WHERE status_6Sigma is null ;

-- 1228
UPDATE `olivier_it` it 
SET status_CMDB = 'KO'
WHERE status_CMDB is null

SELECT * FROM `IT_Equipment_match_position_reduced_v3` it2

INNER JOIN `olivier_it` it 
ON it.reperage_id = it2.reperage_id
and it.nom_de_la_salle = it2.nom_de_la_salle

In [ ]:
-- 1823
UPDATE `olivier_it` it
INNER JOIN `IT_Equipment` it2 
ON it.reperage_id = it2.reperage_id
and it.nom_de_la_salle = it2.nom_de_la_salle

SET it.status_position_6Sigma = "OK"

WHERE  (
   it2.modele not like '%BL460C%'
OR it2.modele not like '%BL660C%'
OR it2.modele not like '%BL20P%'
OR it2.modele not like '%BL860C%'
OR it2.modele not like '%BL420C%'
OR it2.modele not like '%BL680C%'
OR it2.modele not like '%WS460C%'
OR it2.modele not like '%BL870C%' )

In [ ]:
-- 1063
UPDATE `olivier_it` it 
INNER JOIN `assets_filtree_reduced_not_retired`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
    
INNER JOIN `IT_Equipment_match_position_reduced_v3_not_retired` it2
ON it.reperage_id = it2.reperage_id
and it.nom_de_la_salle = it2.nom_de_la_salle

SET it.status_position_CMDB = 'OK'

WHERE  (
   aa.model not like '%BL460C%'
OR aa.model not like '%BL660C%'
OR aa.model not like '%BL20P%'
OR aa.model not like '%BL860C%'
OR aa.model not like '%BL420C%'
OR aa.model not like '%BL680C%'
OR aa.model not like '%WS460C%'
OR aa.model not like '%BL870C%' )

In [ ]:
-- 469
UPDATE `olivier_it` it 
SET status_position_6Sigma = 'KO'
WHERE status_position_6Sigma is null ;

-- 1228
UPDATE `olivier_it` it 
SET status_position_CMDB = 'KO'
WHERE status_position_CMDB is null

## Status hauteur

Check la hauteur des équipements entre CMDB et 6SIGMA

SELECT it.nom, it.Nom_de_la_salle, it.reperage_id, it.status_6Sigma, it.status_position_6Sigma, 
it2.Nom_de_la_salle, it2.reperage_id, it2.hauteur_u, aa.salle, aa.dalle, (aa.slot_end - aa.slot_begin) + 1 hauteur_cmdb
FROM `olivier_it` it

INNER JOIN `IT_Equipment_records` it2 ON it.ASSET_ID = it2.ASSET_ID 

INNER JOIN `assets_filtree_reduced_not_retired` aa 
ON (
	it2.nom_de_la_salle = aa.Salle
AND
	it2.nom_de_baie = aa.Rack
AND
	it2.Slot_U = CONVERT(aa.Slot_Begin, integer)
	
AND ceiling(it2.Hauteur_U) = (aa.slot_end - aa.slot_begin) + 1
) -- delta slot     


WHERE (it.status_6Sigma = 'OK')

UPDATE `olivier_it` it
INNER JOIN `IT_Equipment_records` it2 ON it.ASSET_ID = it2.ASSET_ID 

INNER JOIN `assets_filtree_reduced_not_retired` aa
ON (
	it2.Nom_de_la_salle = aa.Salle
AND
	it2.Nom_de_baie = aa.Rack
AND
	it2.Slot_U = CONVERT(aa.Slot_Begin, integer)
AND 
	ceiling(it2.hauteur_u) = (aa.slot_end - aa.slot_begin) + 1
    ) -- delta slot = hauteur
    
SET it.status_hauteur = 'OK'

WHERE it.status_6

## Equipement non présent sur Olivier_IT

#### Equipement 6sigma non présent sur Olivier_it sur la base de la position (salle, rack, position de départ)

In [ ]:
DROP TABLE IF EXISTS olivier_it_eq_not_in_6sigma ;# MySQL returned an empty result set (i.e. zero rows).
CREATE TABLE olivier_it_eq_not_in_6sigma
as 
select it2.ASSET_ID, it2.nom, it2.Nom_de_la_salle,it2.reperage_id
from `olivier_it` it
RIGHT JOIN `IT_Equipment` it2 
ON it.reperage_id = it2.reperage_id
and it.nom_de_la_salle = it2.nom_de_la_salle

where it.reperage_id is null# 1693 rows affected.
order by reperage_id asc

In [ ]:
DROP TABLE IF EXISTS olivier_it_eq_not_in_cmdb_raw ;
CREATE TABLE olivier_it_eq_not_in_cmdb_raw
as 
select aa.ASSET_ID, aa.nom_court, aa.salle, aa.dalle, aa.reperage_id reperage_id, aa.etat
from `olivier_it` it
RIGHT JOIN `assets_filtree_reduced_not_retired` aa 
ON it.reperage_id = aa.reperage_id
and it.nom_de_la_salle = aa.salle

where it.reperage_id is null and aa.dalle != '' # 1693 rows affected.
order by reperage_id asc

#### NON présent sur CMDB

SELECT * from `olivier_it_eq_not_in_6sigma` it

#### NON présent sur 6SIGMA

DROP VIEW IF EXISTS olivier_it_eq_not_in_cmdb ;
CREATE VIEW olivier_it_eq_not_in_cmdb
AS
SELECT * from `olivier_it_eq_not_in_cmdb_raw` 
where etat != 'Retired' 